In [2]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, ndcg_score


In [3]:
# Chargement et Préparation des Données
import os
from google.colab import drive

# Monter Google Drive pour accéder aux fichiers
drive.mount('/content/drive') # Cette ligne monte votre Google Drive

print(os.getcwd())

data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Movie.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content


In [ ]:
data

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141834,735597,Circles,0.000,0,Released,2015-06-10,0,6,False,NaN,...,Circles,A story of tangled relationships. A couple on ...,0.600,/y7rm1xOISJlNpMo50uGyTEHx94Q.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1141835,735598,Peppa Pig : Gérald La Girafe,0.000,0,Released,NaN,0,0,False,NaN,...,Peppa Pig : Gérald La Girafe,NaN,0.600,/iSWDarmDVBgAm95tnLa4n8oOy6k.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1141836,735601,Não Partimos,0.000,0,Released,2019-01-01,0,5,False,NaN,...,Não Partimos,What is dying? What does happen when we move f...,0.600,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1141837,735602,Caminhos Esquecidos,0.000,0,Released,2019-01-01,0,5,False,NaN,...,Caminhos Esquecidos,How is the stream of consciousness of a person...,0.600,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1141839 entries, 0 to 1141838
Data columns (total 24 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1141839 non-null  int64  
 1   title                 1141826 non-null  object 
 2   vote_average          1141839 non-null  float64
 3   vote_count            1141839 non-null  int64  
 4   status                1141839 non-null  object 
 5   release_date          952763 non-null   object 
 6   revenue               1141839 non-null  int64  
 7   runtime               1141839 non-null  int64  
 8   adult                 1141839 non-null  bool   
 9   backdrop_path         302646 non-null   object 
 10  budget                1141839 non-null  int64  
 11  homepage              120823 non-null   object 
 12  imdb_id               603990 non-null   object 
 13  original_language     1141839 non-null  object 
 14  original_title        1141826 non-

In [ ]:
data.isnull().sum()

,0
id,0
title,13
vote_average,0
vote_count,0
status,0
release_date,189076
revenue,0
runtime,0
adult,0
backdrop_path,839193


In [4]:
# suppression des ligne avec des valeur manquants
data.dropna(inplace=True)
data.isna().sum()

,0
id,0
title,0
vote_average,0
vote_count,0
status,0
release_date,0
revenue,0
runtime,0
adult,0
backdrop_path,0


In [5]:
# 2. Sélection des colonnes nécessaires
columns_to_keep = ['title', 'genres', 'production_countries', 'keywords', 'release_date']
data = data[columns_to_keep]

In [6]:
# Prétraitement : garder les 3 premiers genres et 2 mots-clés
data['genres'] = data['genres'].str.split(',').apply(lambda x: ', '.join(x[:3]) if isinstance(x, list) else '')
data['keywords'] = data['keywords'].str.split(',').apply(lambda x: ', '.join(x[:2]) if isinstance(x, list) else '')

data['release_year'] = data['release_date'].str.split('-').str[0]
data.drop(columns=['release_date'], inplace=True)


data

,title,genres,production_countries,keywords,release_year
0,Inception,"Action, Science Fiction, Adventure","United Kingdom, United States of America","rescue, mission",2010
1,Interstellar,"Adventure, Drama, Science Fiction","United Kingdom, United States of America","rescue, future",2014
2,The Dark Knight,"Drama, Action, Crime","United Kingdom, United States of America","joker, sadism",2008
3,Avatar,"Action, Adventure, Fantasy","United States of America, United Kingdom","future, society",2009
4,The Avengers,"Science Fiction, Action, Adventure",United States of America,"new york city, superhero",2012
...,...,...,...,...,...
1137468,Life After Death: Quantum Realms,Documentary,United States of America,"metaphysics, new age",2017
1137702,The Float,"Drama, Thriller",United Kingdom,"darkness, breaking the fourth wall",2019
1138365,Kilometer 147,Comedy,Israel,"israel, camel",2018
1139122,Sanctuaries of Silence,Documentary,United States of America,"nature, noise pollution",2018


In [7]:
# Créer une colonne combinée pour la vectorisation
data['cat_features'] = data[data.columns].apply(lambda x: ' '.join(x), axis=1)


In [7]:
data

,title,genres,production_countries,keywords,release_year,cat_features
0,Inception,"Action, Science Fiction, Adventure","United Kingdom, United States of America","rescue, mission",2010,"Inception Action, Science Fiction, Adventure..."
1,Interstellar,"Adventure, Drama, Science Fiction","United Kingdom, United States of America","rescue, future",2014,"Interstellar Adventure, Drama, Science Ficti..."
2,The Dark Knight,"Drama, Action, Crime","United Kingdom, United States of America","joker, sadism",2008,"The Dark Knight Drama, Action, Crime United ..."
3,Avatar,"Action, Adventure, Fantasy","United States of America, United Kingdom","future, society",2009,"Avatar Action, Adventure, Fantasy United Sta..."
4,The Avengers,"Science Fiction, Action, Adventure",United States of America,"new york city, superhero",2012,"The Avengers Science Fiction, Action, Advent..."
...,...,...,...,...,...,...
1137468,Life After Death: Quantum Realms,Documentary,United States of America,"metaphysics, new age",2017,Life After Death: Quantum Realms Documentary U...
1137702,The Float,"Drama, Thriller",United Kingdom,"darkness, breaking the fourth wall",2019,"The Float Drama, Thriller United Kingdom dark..."
1138365,Kilometer 147,Comedy,Israel,"israel, camel",2018,"Kilometer 147 Comedy Israel israel, camel 2018"
1139122,Sanctuaries of Silence,Documentary,United States of America,"nature, noise pollution",2018,Sanctuaries of Silence Documentary United Stat...


### Construction du Graphe

In [8]:
# 3. Vectorisation des caractéristiques combinées
vectorizer = CountVectorizer(stop_words='english', min_df=20)
word_matrix = vectorizer.fit_transform(data['cat_features'])

# 4. Calcul de la similarité cosinus
sim = cosine_similarity(word_matrix)


In [16]:
# Créer le graphe
G = nx.Graph()

# Ajouter les nœuds
for idx, row in data.iterrows():
    G.add_node(row['title'], genres=row['genres'], keywords=row['keywords'])

# Ajouter les arêtes avec poids basés sur la similarité
for i in range(len(sim)):
    for j in range(i+1, len(sim)):
        if sim[i][j] > 0.5:  # Seulement les films avec une similarité supérieure à un seuil
            G.add_edge(data.iloc[i]['title'], data.iloc[j]['title'], weight=sim[i][j])

# Afficher quelques informations sur le graphe
print(f"Nombre de films (nœuds) : {G.number_of_nodes()}")
print(f"Nombre de relations similaires (arêtes) : {G.number_of_edges()}")

Nombre de films (nœuds) : 9498
Nombre de relations similaires (arêtes) : 3050909


Recommandation par Graphe

In [17]:
# 6. Fonction de recommandation basée sur les graphes
def get_recom_graph(titre, G, count=10):
    """
    Obtenir des recommandations basées sur un graphe de similarité.

    Parameters:
    - titre : str, titre du film pour lequel des recommandations sont recherchées.
    - G : NetworkX Graph, le graphe des films et leurs relations similaires.
    - count : int, nombre de recommandations souhaitées (par défaut 10).

    Returns:
    - list : titres des films recommandés.
    """
    if titre not in G:
        return []

    # Récupérer les voisins (films similaires)
    voisins = sorted(G[titre].items(), key=lambda x: x[1]['weight'], reverse=True)

    # Obtenir les meilleurs résultats, en excluant l'élément lui-même
    top_rec = voisins[:count]

    # Générer la liste des titres
    titres = [rec[0] for rec in top_rec]
    return titres

# Exemple d'application
recommandations = get_recom_graph(titre='The Avengers', G=G, count=9)
print(recommandations)


['The Incredible Hulk', 'Teenage Mutant Ninja Turtles', 'Captain America: The First Avenger', 'Escape from New York', 'Spider-Man: Homecoming', 'The Time Machine', 'X2', 'John Carter', 'Meet Dave']


Évaluation du Modèle

In [18]:
# Diviser les données en ensemble d'entraînement et de test
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Créer un ensemble de "films réels" pour chaque film du test (ceux qui sont réellement similaires)
real_recommendations = {}

for idx, row in test_data.iterrows():
    titre = row['title']
    voisins = get_recom_graph(titre, G, count=5)  # Obtenez 5 recommandations pour chaque film du test
    real_recommendations[titre] = voisins

In [19]:
# Évaluation avec des métriques classiques : Précision, Rappel et F1

def evaluate_recommendations(true_recommendations, predicted_recommendations):
    precision_scores = []
    recall_scores = []
    f1_scores = []

    for titre, true_recs in true_recommendations.items():
        if titre in predicted_recommendations:
            pred_recs = predicted_recommendations[titre]
            # Calcul de la précision, rappel et F1
            intersection = len(set(true_recs).intersection(set(pred_recs)))
            precision = intersection / len(pred_recs) if len(pred_recs) > 0 else 0
            recall = intersection / len(true_recs) if len(true_recs) > 0 else 0
            f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

            precision_scores.append(precision)
            recall_scores.append(recall)
            f1_scores.append(f1)

    return np.mean(precision_scores), np.mean(recall_scores), np.mean(f1_scores)


In [20]:
# Calcul des recommandations prédites pour l'évaluation
predicted_recommendations = {}
for titre in real_recommendations.keys():
    predicted_recommendations[titre] = get_recom_graph(titre, G, count=5)

precision, recall, f1 = evaluate_recommendations(real_recommendations, predicted_recommendations)
print(f"Précision: {precision:.4f}, Rappel: {recall:.4f}, F1: {f1:.4f}")

Précision: 0.9943, Rappel: 0.9943, F1: 0.9943
